In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path

import numpy as np
import torch
import torch as tr
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
from torchvision.transforms import v2
import pandas as pd
import matplotlib.pyplot as plt
import mplcyberpunk

plt.style.use("cyberpunk")

%matplotlib inline

In [39]:
from impl.arch import *

In [4]:
import impl.retinal_dataset as rd

transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Resize((224, 224)),
])

train_dataset = rd.RetinalDataset(transform=transform)

100%|█████████████████████████████| 512/512 [01:20<00:00,  6.33it/s]


In [57]:
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True)

In [ ]:
# plt.figure(figsize=(12,4))
# disease_counts = train_dataset.labels_df.iloc[:, 2:].sum().sort_values(ascending=False)
# disease_counts.plot.bar()

In [58]:
model = Model1()

In [60]:
optimizer = torch.optim.Adam(model.parameters(), lr=100)
loss_fn = nn.BCEWithLogitsLoss()

In [61]:
for i in range(8):
    epoch_loss = 0
    b = 0

    for images, labels in train_dataloader:
        model.train()

        optimizer.zero_grad()
        logits = model(images)
        probs = F.sigmoid(logits)
        loss = loss_fn(probs, labels.view(-1, 1).float())
        print(f"{b+1}: {loss}")

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        b += 1

    print(f"==== Epoch {i+1}: train_loss={epoch_loss/len(train_dataset):.10f}")

print("Done")

1: 0.5525802969932556
2: 0.4616992473602295
3: 0.4695117175579071
4: 0.4304492473602295
==== Epoch 1: train_loss=0.0037387510
1: 0.4460742473602295
2: 0.4538867473602295
3: 0.4460742175579071
4: 0.4851367175579071
==== Epoch 2: train_loss=0.0035765077
1: 0.4382617175579071
2: 0.4538867175579071
3: 0.4851367473602295
4: 0.4538867473602295
==== Epoch 3: train_loss=0.0035765077
1: 0.4695117175579071
2: 0.4851367175579071
3: 0.4382617175579071
4: 0.4382617175579071
==== Epoch 4: train_loss=0.0035765076
1: 0.4460742473602295
2: 0.4695117473602295
3: 0.4695117175579071
4: 0.4460742473602295
==== Epoch 5: train_loss=0.0035765077
1: 0.4695117175579071
2: 0.4460742175579071
3: 0.4616992473602295
4: 0.4538867175579071
==== Epoch 6: train_loss=0.0035765076
1: 0.4304492175579071
2: 0.4382617175579071
3: 0.4616992473602295
4: 0.5007616877555847
==== Epoch 7: train_loss=0.0035765076
1: 0.5241992473602295
2: 0.4148242175579071
3: 0.4226367175579071
4: 0.4695117175579071
==== Epoch 8: train_loss=0.003

In [62]:
# torch.save({'model_state': model.state_dict()},'checkpoints/model1_001.chk')

In [ ]:
# checkpoint = torch.load('checkpoints/model1_001.chk')
# model.load_state_dict(checkpoint['model_state'])